## CIFAR 10

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *
PATH = "../fp16/data/cifar10/"
os.makedirs(PATH,exist_ok=True)

In [3]:
from fp16util import *

### Load Data

In [4]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

In [5]:
def get_data(sz,bs):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
    return ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [6]:
bs=128

## Initial model

In [ ]:
%pdb on

In [7]:
from fastai.models.cifar10.resnext import resnext29_8_64

m = resnext29_8_64()
bm = BasicModel(m.cuda(), name='cifar10_rn29_8_64')
data = get_data(8,bs*4)

In [8]:
learn = ConvLearner(data, bm)
learn.unfreeze()
lr=1e-2; wd=5e-4

In [9]:
learn.half()

In [ ]:
learn.load('fp32_model')

In [10]:
%time learn.fit(lr, 1, cycle_len=.5, loss_scale=128, fp16=True)

50it [00:35,  1.42it/s, loss=2.24]                        epoch      trn_loss   val_loss   accuracy   
    0      2.236042   2.433496   0.329354  

CPU times: user 42.2 s, sys: 11 s, total: 53.2 s
Wall time: 38.1 s


[2.43349609375, 0.3293543204665184]

In [ ]:
learn.model

In [ ]:
learn.save('fp16_model')

## Testing 32

In [ ]:
from fastai.models.cifar10.resnext import resnext29_8_64

m = resnext29_8_64()
bm = BasicModel(m.cuda(), name='cifar10_rn29_8_64')
data = get_data(8,bs*4)

In [ ]:
learn = ConvLearner(data, bm)
learn.unfreeze()
lr=1e-2; wd=5e-4

In [ ]:
learn.load('fp16_model')

In [ ]:
%time learn.fit(lr, 1, cycle_len=.5)

In [ ]:
learn.save('fp32_model')